<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# NFTD downsampling and reprojection

_by Jeremy Freeman (CarbonPlan), August 2, 2020_

This notebook converts NFTD 250m yearly rasters stored in Cloud Optimized GeoTIFF
into 400m GeoTIFFs.

**Inputs:**

- COG outputs from `01_nftd_to_cogs.ipynb`

**Outputs:**

- 1 Zarr archive:
  `gs://carbonplan-data/processed/MTBS/raster.zarr/<RESOLUTION>/<VAR>`

**Notes:**

- In the process of processing this dataset, we found that the behavior in
  rasterio's `reproject` function was sensitive to the package version for
  rasterio and/or gdal. Versions we found to work were
  `rasterio=1.0.25,gdal=2.4.2`. Versions that we found to fail were
  `rasterio=1.1.5,gdal=3.1.0`
